In [14]:
import pandas as pd
import os

In [15]:
data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/raw_data/'
cleaned_data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/cleaned_data/'
variables_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/variables/'

In [16]:
frims_rc_id_mapping = pd.read_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/frims_rc_id_mapping.csv')
frims_rc_id_mapping = frims_rc_id_mapping[['object_id','RC_Id']]

In [17]:
df = pd.read_excel(data_path+'FRIMS_Inf_Damage_Data_21_july_2023.xlsx',
                   sheet_name=0)

In [21]:
df = df.rename(columns = {'Revenue Circle Id': 'RC_Id'})

In [22]:
df = df.merge(frims_rc_id_mapping, on='RC_Id', how='left')

In [23]:
unmapped_rcs = set(df[df['object_id'].isnull()]['RC_Id'].dropna().to_list())
unmapped_rcs

set()

In [24]:
df['month'] = df['Report Date'].dt.month
df['month'] = df.month.map("{:02}".format)
df['year'] = df['Report Date'].dt.year

In [26]:
infra_damages_df = df.groupby(['year', 'month', 'object_id', 'Type'])[['Id']].count().reset_index()

In [27]:
infra_damages_df.columns = ['year', 'month', 'object_id', 'Type', 'num_damages']

In [28]:
infra_damages_df['year_month'] = infra_damages_df['year'].astype(str) + '_' + infra_damages_df['month'].astype(str)

In [29]:
variables = infra_damages_df.Type.unique()

In [30]:
for variable in variables:
    variable_df = infra_damages_df[infra_damages_df['Type'] == variable]
    
    for year_month in variable_df['year_month'].unique():
        variable_df_monthly = variable_df[variable_df['year_month']==year_month]  
        
        variable_df_monthly = variable_df_monthly.rename(columns = {'num_damages': variable})
        variable_df_monthly = variable_df_monthly[['object_id', variable]]
        if os.path.exists(variables_path+variable):
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)
        else:
            os.mkdir(variables_path+variable)
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)

In [ ]:
df[df['Revenue Circle Id']==61]

In [ ]:
frims_rc_id_mapping[frims_rc_id_mapping['RC_Id']==169]